In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
crashes = pd.read_pickle('crashes_roads_segments.pkl')

In [3]:
crashes.head()

,CRASHID,RouteName,BeginMilePost,RoadView,CrashSeverity,MANNERCOLLISIONID,ROADWAYJUNCTFEATUREID,NUMBERVEHICLESINVOLVED,FIRSTHARMFULEVENTID,ROADWAYTYPECD,ROUTEDIRECTION,EXITNUMBER,RAMPID,Latitude,Longitude,db,project,run,lat,lon,dayofyear,year,month,day,hour,dayofweek,incident,LABEL,AVG_AADT,AvE_CRT,AvE_SEV_CR,BEG_MP,CRT,C_MI,DESCRIP,END_MP,EXP_CRT,EXP_SEV_CR,FC_NAME,OBJECTID,PROTECTED,REGION,RouteNm,SAF_INDEX,SEV_CRT,SEV_C_MI,State_FA,TOT_C,TOT_SEV_C,SHAPE
11,10401151,0009P,11.64,http://168.178.125.111/virtualnavigator/udot/V...,2,Front to Rear,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119389.0,298229.0,2014,0009,P,37.199177,-113.273482,288,2010,10,15,19,4,1,0009P,11829.619048,0.287125,0.000000,10.883,0.786722,3.397028,860 N (Enchanted Way) via State St - 300 S La ...,12.458,2.74,6.8,Urban Principal Arterial,2045,23 USC 409,4,SR-9,2.0,0.000000,0.000000,State,16,0,"{'paths': [[[-12610092.850200001, 4465474.1281..."
23,10394697,0009P,11.85,http://168.178.125.111/virtualnavigator/udot/V...,1,Sideswipe Same Direction,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119704.0,298349.0,2014,0009,P,37.202040,-113.272216,232,2010,8,20,19,4,1,0009P,11829.619048,0.287125,0.000000,10.883,0.786722,3.397028,860 N (Enchanted Way) via State St - 300 S La ...,12.458,2.74,6.8,Urban Principal Arterial,2045,23 USC 409,4,SR-9,2.0,0.000000,0.000000,State,16,0,"{'paths': [[[-12610092.850200001, 4465474.1281..."
33,10346333,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Delineator Post,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,86,2010,3,27,21,5,1,0009P,3031.666667,0.463938,0.000000,13.282,0.626316,0.693132,100 East La Verkin - Mill St Virgin,18.580,1.35,5.1,Rural Principal Arterial,1631,23 USC 409,4,SR-9,1.0,0.000000,0.000000,State,11,0,"{'paths': [[[-12608433.0227, 4469426.418499999..."
45,10352753,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Overturn/Rollover,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,106,2010,4,16,13,4,1,0009P,3031.666667,0.463938,0.000000,13.282,0.626316,0.693132,100 East La Verkin - Mill St Virgin,18.580,1.35,5.1,Rural Principal Arterial,1631,23 USC 409,4,SR-9,1.0,0.000000,0.000000,State,11,0,"{'paths': [[[-12608433.0227, 4469426.418499999..."
49,10357506,0009P,21.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Animal - Wild,Mainline,P,NaN,NaN,4118637.0,309403.0,2014,0009,P,37.194752,-113.147468,64,2010,3,5,9,4,1,0009P,1881.666667,1.242143,2.055015,18.580,1.676893,1.151908,Mill Street Virgin - Center St Rockville,27.847,1.35,5.1,Rural Principal Arterial,375,23 USC 409,4,SR-9,5.0,10.480579,0.071994,State,32,2,"{'paths': [[[-12600111.84, 4467519.9580999985]..."


In [4]:
weatherstn = pd.read_pickle('HourlyWeatherActiveStations.pkl')
weatherstn.head()

,Station ID,Station Name,Latitude,Longitude,Elevation,County,Mesonet,First Data Date,Last Data Date,Status,firstdate,lastdate
25,ACRU1,AGUA CANYON,37.522222,-112.271111,8890,Kane,RAWS,1997-01-08,2018-01-27,ACTIVE,1997-01-08,2018-01-27
27,AGD,ALTA - GUARD HOUSE,40.5905,-111.638,8799,Salt Lake,AVALANCHE,1998-11-25,2018-01-27,ACTIVE,1998-11-25,2018-01-27
29,AGUU1,AGUA CANYON,37.522170,-112.271180,8889,Kane,SNOTEL,2000-01-20,2018-01-27,ACTIVE,2000-01-20,2018-01-27
33,ALKU1,ALKALI MESA,37.67455,-109.36429,6444,San Juan,SCAN,2008-03-06,2018-01-27,ACTIVE,2008-03-06,2018-01-27
35,ALT,ALTA - TOP OF COLLINS,40.571,-111.631,10443,Salt Lake,AVALANCHE,1997-01-01,2018-01-27,ACTIVE,1997-01-01,2018-01-27


In [5]:
weatherstn.dtypes

Station ID                 object
Station Name               object
Latitude                   object
Longitude                  object
Elevation                  object
County                     object
Mesonet                    object
First Data Date            object
Last Data Date             object
Status                     object
firstdate          datetime64[ns]
lastdate           datetime64[ns]
dtype: object

In [6]:
cols = ['Latitude', 'Longitude']
weatherstn[cols] = weatherstn[cols].apply(pd.to_numeric, errors='coerce')

In [7]:
weather_coords = weatherstn[['Latitude', 'Longitude']].values[np.newaxis, :]           # airport=weather, places = crashes
crashes_coords = np.rollaxis(crashes[['lat', 'lon']].values[np.newaxis, :], 0, -1)

In [8]:
dist_coords = ((crashes_coords - weather_coords)**2).sum(axis=-1)
min_coords = dist_coords.argmin(axis=1)

crashes_coords.shape, weather_coords.shape, dist_coords.shape, min_coords.shape

((342748, 1, 2), (1, 474, 2), (342748, 474), (342748,))

In [9]:
merge_data = crashes.copy()

print(merge_data.shape)

merge_data['weatherstn_index'] = weatherstn.index[min_coords]

(342748, 50)


In [10]:
df = weatherstn.loc[merge_data.weatherstn_index, ['Station ID', 'Station Name']]
merge_data[['Station ID', 'Station Name']] = df.set_index(merge_data.index)

print(merge_data.shape)

(342748, 53)


In [11]:
merge_data.head()

,CRASHID,RouteName,BeginMilePost,RoadView,CrashSeverity,MANNERCOLLISIONID,ROADWAYJUNCTFEATUREID,NUMBERVEHICLESINVOLVED,FIRSTHARMFULEVENTID,ROADWAYTYPECD,ROUTEDIRECTION,EXITNUMBER,RAMPID,Latitude,Longitude,db,project,run,lat,lon,dayofyear,year,month,day,hour,dayofweek,incident,LABEL,AVG_AADT,AvE_CRT,AvE_SEV_CR,BEG_MP,CRT,C_MI,DESCRIP,END_MP,EXP_CRT,EXP_SEV_CR,FC_NAME,OBJECTID,PROTECTED,REGION,RouteNm,SAF_INDEX,SEV_CRT,SEV_C_MI,State_FA,TOT_C,TOT_SEV_C,SHAPE,weatherstn_index,Station ID,Station Name
11,10401151,0009P,11.64,http://168.178.125.111/virtualnavigator/udot/V...,2,Front to Rear,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119389.0,298229.0,2014,0009,P,37.199177,-113.273482,288,2010,10,15,19,4,1,0009P,11829.619048,0.287125,0.000000,10.883,0.786722,3.397028,860 N (Enchanted Way) via State St - 300 S La ...,12.458,2.74,6.8,Urban Principal Arterial,2045,23 USC 409,4,SR-9,2.0,0.000000,0.000000,State,16,0,"{'paths': [[[-12610092.850200001, 4465474.1281...",3843,WRRU1,WHITE REEF
23,10394697,0009P,11.85,http://168.178.125.111/virtualnavigator/udot/V...,1,Sideswipe Same Direction,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119704.0,298349.0,2014,0009,P,37.202040,-113.272216,232,2010,8,20,19,4,1,0009P,11829.619048,0.287125,0.000000,10.883,0.786722,3.397028,860 N (Enchanted Way) via State St - 300 S La ...,12.458,2.74,6.8,Urban Principal Arterial,2045,23 USC 409,4,SR-9,2.0,0.000000,0.000000,State,16,0,"{'paths': [[[-12610092.850200001, 4465474.1281...",3843,WRRU1,WHITE REEF
33,10346333,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Delineator Post,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,86,2010,3,27,21,5,1,0009P,3031.666667,0.463938,0.000000,13.282,0.626316,0.693132,100 East La Verkin - Mill St Virgin,18.580,1.35,5.1,Rural Principal Arterial,1631,23 USC 409,4,SR-9,1.0,0.000000,0.000000,State,11,0,"{'paths': [[[-12608433.0227, 4469426.418499999...",3843,WRRU1,WHITE REEF
45,10352753,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Overturn/Rollover,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,106,2010,4,16,13,4,1,0009P,3031.666667,0.463938,0.000000,13.282,0.626316,0.693132,100 East La Verkin - Mill St Virgin,18.580,1.35,5.1,Rural Principal Arterial,1631,23 USC 409,4,SR-9,1.0,0.000000,0.000000,State,11,0,"{'paths': [[[-12608433.0227, 4469426.418499999...",3843,WRRU1,WHITE REEF
49,10357506,0009P,21.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Animal - Wild,Mainline,P,NaN,NaN,4118637.0,309403.0,2014,0009,P,37.194752,-113.147468,64,2010,3,5,9,4,1,0009P,1881.666667,1.242143,2.055015,18.580,1.676893,1.151908,Mill Street Virgin - Center St Rockville,27.847,1.35,5.1,Rural Principal Arterial,375,23 USC 409,4,SR-9,5.0,10.480579,0.071994,State,32,2,"{'paths': [[[-12600111.84, 4467519.9580999985]...",3863,ZIOU1,ZION CANYON


In [12]:
merge_data.to_pickle('crashes_weatherstn.pkl')